# Imports

In [1]:
import pandas as pd; import os; from analysis_helpers import *; import warnings
warnings.filterwarnings('ignore')

# Data Compile
Organize behavioral data from each participant into one dataframe for the whole experiment.

In [8]:
# Exp1 and exp2 -- group 1 timing safety
    
    # run simulations with psychopy code
    
    # make safest possible estimate
    
# Exp2 cue correction

    # exp simulate
    
    # correct all labels
    
    # do a manual check

In [6]:
# Compile the behavioral and gaze data for both exps, Groups 1 and 2

    # for each subj:
        
        # compile behavioral
        
            # pull from log files and convert times as needed
        
        # check gaze data (file for each, data for all times needed, units, etc)
        
        # check for redundancies
        
        # do gaze calcs and add to dataframe
        
        
        # save out compiled full csv in this subject's dir (under temp)
        

In [ ]:
# compile data from all temps, all experiments

In [5]:
# Compile the behavioral data for both exps, Groups 1 and 2

# Complete all necessary checks

# 

# Produce a single dataframe with all of the necessary data, labeled by: 
# experiment and group number



data_dirs = ['../sustained_attention_experiment/data',
             '../variable_attention_experiment/data',
            '',
            '']



filenames = ['../parsed_data/behavioral_data_sustained.csv', 
             '../parsed_data/behavioral_data_variable.csv']

for data,file in zip(data_dirs, filenames):
    
    sub_list = []
    
    for sub_dir in os.listdir(data):
        sub_list.append(add_level(sum_pd(data + '/' + sub_dir)))

    exp_raw = pd.concat(sub_list)
    exp_raw.to_csv(file)

NotADirectoryError: [Errno 20] Not a directory: '../sustained_attention_experiment/data/.DS_Store'